In [2]:
import struct
import numpy as np

def write_bin(filename, array):
    # Force endianess: https://stackoverflow.com/questions/23831422/what-endianness-does-python-use-to-write-into-files
    dtype_to_format = {
        np.int: 'i',
        np.int8: 'i',
        np.int16: 'i',
        np.int32: 'i',
        np.int64: 'i',
        np.unsignedinteger: 'I',
        np.float: 'f',
        np.float16: 'f',
        np.float32: 'f',
        np.float64: 'f',
        np.double: 'd'
    }
    fmt = dtype_to_format[array.dtype.type]
    row, col = array.shape
    with open(filename, "wb") as f:
        f.write(struct.pack('i', row))
        f.write(struct.pack('i', col))
        f.write(struct.pack('c', bytes(fmt, 'utf-8')))
        f.write(struct.pack(f"{fmt}"*(row * col), *x.flatten(order="C").tolist()))

def read_bin(filename):
    # https://qiita.com/madaikiteruyo/items/dadc99aa29f7eae0cdd0
    format_to_byte = {
        'c': 1,
        'i': 4,
        'I': 4,
        'f': 4,
        'd': 8
    }

    data = []
    dims, fmt = None, None
    with open(filename, "rb") as f:
        # read row and col (np.int = 4 bytes)
        byte = f.read(2 * format_to_byte['i'])
        if byte == b'':
            raise Exception("read_bin: Empty binary")
        else:
            dims = struct.unpack('ii', byte)
        
        if len(dims) != 2: raise Exception("read_bin: No dimensions (row, col) dumped in binary")

        # Read character format
        byte = f.read(1)
        if byte == b'':
            raise Exception("read_bin: Empty binary")
        else:
            fmt = chr(struct.unpack('c', byte)[0][0])

        if len(fmt) != 1: raise Exception("read_bin: No format dumped in binary")
        
        while True:
            byte = f.read(format_to_byte[fmt])
            if byte == b'':
                break
            else:
                data.append(struct.unpack(fmt, byte)[0])
        
    return np.array(data).reshape(dims[0], dims[1])

x = np.arange(1, 11).reshape(2, 5)
print(x)
write_bin("array.bin", x)
read_bin("array.bin")


[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]


array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10]])

In [33]:
# read in cpp https://github.com/3outeille/cpp-experiments/blob/master/stl/map_of_files/read_files.cpp
# read in python: https://qiita.com/madaikiteruyo/items/dadc99aa29f7eae0cdd0